In [1]:
import pymc as pm
import pandas as pd
import numpy as np
import aesara.tensor as at
import pytensor.tensor as pt
import ast
import random
import math

WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl


C:\Users\ionut\anaconda3\envs\pymc_env\Lib\site-packages\numpy\distutils\system_info.py:2159: UserWarning: 
    Optimized (vendor) Blas libraries are not found.
    Falls back to netlib Blas library which has worse performance.
    A better performance should be easily gained by switching
    Blas library.
  if self._calc_info(blas):
C:\Users\ionut\anaconda3\envs\pymc_env\Lib\site-packages\numpy\distutils\system_info.py:2159: UserWarning: 
    Blas (http://www.netlib.org/blas/) libraries not found.
    Directories to search for the libraries can be specified in the
    numpy/distutils/site.cfg file (section [blas]) or by setting
    the BLAS environment variable.
  if self._calc_info(blas):
C:\Users\ionut\anaconda3\envs\pymc_env\Lib\site-packages\numpy\distutils\system_info.py:2159: UserWarning: 
    Blas (http://www.netlib.org/blas/) sources not found.
    Directories to search for the sources can be specified in the
    numpy/distutils/site.cfg file (section [blas_src]) or by setting

In [2]:
ratings = pd.read_csv("./ratings.csv")

In [3]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [4]:
movies = pd.read_csv("./movies_metadata.csv")
movies.head(60)

C:\Users\ionut\AppData\Local\Temp\ipykernel_17136\3389714093.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("./movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [5]:
mainGenres = []


for i,row in movies.iterrows():

    if ast.literal_eval(row['genres'])==[]:
        mainGenres.append("")
        continue 
        
    x = ast.literal_eval(row['genres'])[0]['name']
    mainGenres.append(x)

In [6]:
movies['mainGenre'] = mainGenres

In [7]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,mainGenre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Animation
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Adventure
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Romance
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Comedy
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Comedy


In [8]:
N_movies = 200
selected_movies = []

proportions = []

for genre in set(mainGenres):
    if genre == '' or genre=='Carousel Productions' or genre=='Aniplex' or genre =='Odyssey Media':
        continue

    prop = round(N_movies * list(movies['mainGenre']).count(genre)/len(list(movies['mainGenre'])))

    proportions.append([genre,prop])    

In [9]:
currentlyChosen=sum(x[1] for x in proportions)

while(currentlyChosen<N_movies):

    weights = np.array([1/x[1] if x[1] else 1e9 for x in proportions])
    weights /= weights.sum()
   # print(weights)
   # break
    x = random.choices(list(range(len(proportions))), weights=weights, k=1)[0]


    currentlyChosen+=1 

    proportions[x][1]+=1

In [10]:
propDict = dict()

for x in proportions:
    propDict[x[0]] = x[1]

propDict

{'Animation': 5,
 'Family': 3,
 'Action': 20,
 'Music': 2,
 'Fantasy': 4,
 'History': 2,
 'Crime': 7,
 'Adventure': 7,
 'Thriller': 7,
 'Science Fiction': 4,
 'Drama': 53,
 'War': 4,
 'Documentary': 15,
 'Mystery': 4,
 'Western': 3,
 'Foreign': 1,
 'Romance': 6,
 'TV Movie': 2,
 'Horror': 12,
 'Comedy': 39}

In [11]:
movieId_counts = ratings.groupby('movieId').size()

In [12]:
movieId_counts = ratings.groupby('movieId')['movieId'].count().sort_values(ascending=False)

In [13]:
movieId_counts.head(10)

movieId
356     91921
318     91082
296     87901
593     84078
2571    77960
260     77045
480     74355
527     67662
110     66512
1       66008
Name: movieId, dtype: int64

In [14]:
movies["id"]

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45466, dtype: object

In [15]:
genre_counts = movies['mainGenre'].value_counts()

movies['genre_frequency'] = movies['mainGenre'].map(genre_counts)


movies_sorted = movies.sort_values(by='genre_frequency')

In [16]:
movies_sorted.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,mainGenre,genre_frequency
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aniplex,1
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carousel Productions,1
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Odyssey Media,1
36837,False,NaN,0,"[{'id': 10769, 'name': 'Foreign'}]",NaN,33130,tt1532951,en,Gewoon Hans,No overview found.,...,0.0,"[{'iso_639_1': 'nl', 'name': 'Nederlands'}]",Released,NaN,Gewoon Hans,False,7.0,1.0,Foreign,118
44047,False,NaN,0,"[{'id': 10769, 'name': 'Foreign'}, {'id': 18, ...",NaN,40886,tt0117338,ja,PiCNiC,Believing that the world will end that very da...,...,72.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Picnic,False,6.2,12.0,Foreign,118


In [17]:


for movieId,cnt in movieId_counts.items():
    #print(str(movieId))
    mainGenre = movies_sorted[movies_sorted["id"]==str(movieId)]["mainGenre"]
    #print(mainGenre)
    #break
    if len(mainGenre) == 0:
        continue
    else:
        mainGenre = list(mainGenre)[0]

    if mainGenre not in propDict.keys():
        continue
    if propDict[mainGenre]>0:
        propDict[mainGenre]-=1 
        selected_movies.append(movieId)

        if len(selected_movies)==N_movies:
            break
    print(propDict)
    #print(mainGenre)
    #print(propDict)
    #break
    

{'Animation': 5, 'Family': 3, 'Action': 20, 'Music': 2, 'Fantasy': 4, 'History': 2, 'Crime': 7, 'Adventure': 7, 'Thriller': 7, 'Science Fiction': 4, 'Drama': 52, 'War': 4, 'Documentary': 15, 'Mystery': 4, 'Western': 3, 'Foreign': 1, 'Romance': 6, 'TV Movie': 2, 'Horror': 12, 'Comedy': 39}
{'Animation': 5, 'Family': 3, 'Action': 19, 'Music': 2, 'Fantasy': 4, 'History': 2, 'Crime': 7, 'Adventure': 7, 'Thriller': 7, 'Science Fiction': 4, 'Drama': 52, 'War': 4, 'Documentary': 15, 'Mystery': 4, 'Western': 3, 'Foreign': 1, 'Romance': 6, 'TV Movie': 2, 'Horror': 12, 'Comedy': 39}
{'Animation': 5, 'Family': 3, 'Action': 19, 'Music': 2, 'Fantasy': 4, 'History': 2, 'Crime': 7, 'Adventure': 7, 'Thriller': 7, 'Science Fiction': 4, 'Drama': 51, 'War': 4, 'Documentary': 15, 'Mystery': 4, 'Western': 3, 'Foreign': 1, 'Romance': 6, 'TV Movie': 2, 'Horror': 12, 'Comedy': 39}
{'Animation': 5, 'Family': 3, 'Action': 18, 'Music': 2, 'Fantasy': 4, 'History': 2, 'Crime': 7, 'Adventure': 7, 'Thriller': 7, 'Sc

In [21]:
#selected_movies.append(40886)
#,40886)

In [18]:
len(selected_movies)

200

In [19]:
ratings2 = ratings

In [20]:
ratings2['score'] = ratings2["movieId"].isin(selected_movies).astype(int)

In [21]:
scores= ratings2.groupby('userId')['score'].sum()

scores

userId
1           8
2          11
3           5
4          12
5           5
         ... 
270892     30
270893     28
270894     40
270895     17
270896    101
Name: score, Length: 270896, dtype: int32

In [22]:
N_users = 200

selected_users = []

lower_bound = math.floor(N_movies*0.35)
upper_bound = math.ceil(N_movies*0.65)

for x,y in scores.items():

    if y>=lower_bound and y<=upper_bound:
        selected_users.append(x)

In [23]:
len(selected_users)

13862

In [24]:
good_users = []
watched = np.zeros(N_movies)



In [25]:
for i in range(N_users):
    
    scores = np.zeros(len(selected_users))
    
    for j in range(N_movies):
        if watched[j]:
            continue
        exists = ((ratings['movieId'] == selected_movies[j]) & (ratings['userId'] == selected_users[i])).any()
        scores[i]+=exists

    l = [list(x) for x in zip(selected_users,scores)]
    #print(l)
    l.sort(key = lambda x:-x[1])


    good_users.append(l[0][0])

    for j in range(N_movies):
        exists = ((ratings['movieId'] == selected_movies[j]) & (ratings['userId'] == l[0][0])).any()
        if exists:
            watched[j]=1
    selected_users = [x[0] for x in l[1:]]
    

In [26]:
selected_users = [x for x in good_users]

In [27]:
sanityCheck = ratings[(ratings['movieId'].isin(selected_movies) & ratings['userId'].isin(selected_users))].drop_duplicates(subset=['movieId', 'userId'])

In [28]:
sanityCheck.head()

,userId,movieId,rating,timestamp,score
1711,24,2,3.0,979974023,1
1712,24,6,4.0,979870499,1
1713,24,16,3.0,979870379,1
1714,24,17,3.0,979974163,1
1715,24,21,4.0,979870629,1


In [29]:
#interactions_df.to_csv('sanity_check.csv')

In [30]:
user_id_mapping = {old: new for new, old in enumerate(sorted(sanityCheck['userId'].unique()))}
movie_id_mapping = {old: new for new, old in enumerate(sorted(sanityCheck['movieId'].unique()))}

In [31]:
sanityCheck['userId'] = sanityCheck['userId'].map(user_id_mapping)
sanityCheck['movieId'] = sanityCheck['movieId'].map(movie_id_mapping)

sanityCheck.to_csv('full_dataset.csv')

In [32]:
len(selected_users)
#len(selected_movies)

200